In [1]:
"""
02_Time_Series_Insights.py  —  Time-Series Insights
=====================================================
Education Spending Analytics  |  U.S. K-12 Finance & NAEP Scores 1992-2019

Dependencies:  numpy  pandas  matplotlib  (no seaborn, no scipy, no sklearn)

Run from the project root  OR  from inside a Jupyter cell after adjusting ROOT.
All figures are saved to  outputs/figures/ts/
"""

# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import warnings, math
from pathlib import Path

import matplotlib
matplotlib.use("Agg")          # swap to "TkAgg" / remove for interactive use

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors
import matplotlib.cm as mcm
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# ── Paths (Jupyter-safe) ───────────────────────────────────────────────────
ROOT     = Path.cwd().parent
RAW_PATH = ROOT / "Education Funding Analytics" / "data" / "raw" / "states_all.csv"
FIG_DIR  = ROOT / "Education Funding Analytics" / "outputs" / "figures" / "ts"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# ── Grade-school colour palette ────────────────────────────────────────────
BLUE   = "#1565C0"
RED    = "#D32F2F"
GREEN  = "#2E7D32"
YELLOW = "#F9A825"
ORANGE = "#E65100"
PURPLE = "#6A1B9A"
BLACK  = "#212121"

# Four spotlight states always use the same colour
SPOTLIGHT_STATES  = ["PENNSYLVANIA", "ALABAMA", "CALIFORNIA", "NEBRASKA"]
SPOTLIGHT_LABELS  = ["Pennsylvania (PA)", "Alabama (AL)", "California (CA)", "Nebraska (NE)"]
SPOTLIGHT_COLORS  = [BLUE, RED, GREEN, ORANGE]
SPOTLIGHT_MARKERS = ["o", "s", "^", "D"]

SCHOOL_CYCLE = [BLUE, RED, GREEN, YELLOW, ORANGE, PURPLE, BLACK]

# ── Plot style ─────────────────────────────────────────────────────────────
plt.rcParams.update({
    "figure.dpi"       : 130,
    "figure.facecolor" : "#FFFDE7",      # warm cream — like old notebook paper
    "axes.facecolor"   : "#FFFDE7",
    "axes.titlesize"   : 14,
    "axes.labelsize"   : 12,
    "axes.edgecolor"   : BLACK,
    "axes.linewidth"   : 1.2,
    "axes.grid"        : True,
    "grid.color"       : "#E0E0E0",
    "grid.linewidth"   : 0.8,
    "legend.fontsize"  : 9,
    "xtick.labelsize"  : 10,
    "ytick.labelsize"  : 10,
    "xtick.color"      : BLACK,
    "ytick.color"      : BLACK,
    "text.color"       : BLACK,
    "font.family"      : "DejaVu Sans",
})


def save(fig: plt.Figure, name: str) -> None:
    fig.savefig(FIG_DIR / name, bbox_inches="tight", facecolor=fig.get_facecolor())
    plt.close(fig)
    print(f"  ✓  Saved  outputs/figures/ts/{name}")


# =============================================================================
# PURE-NUMPY STATISTICAL HELPERS
# =============================================================================

def _ols(x: np.ndarray, y: np.ndarray):
    """Return (slope, intercept, r, p, se) via numpy OLS."""
    x, y = np.asarray(x, float), np.asarray(y, float)
    ok   = np.isfinite(x) & np.isfinite(y)
    x, y = x[ok], y[ok]
    n = len(x)
    if n < 3:
        return tuple([np.nan] * 5)
    xm, ym = x.mean(), y.mean()
    ssxx = ((x - xm) ** 2).sum()
    ssyy = ((y - ym) ** 2).sum()
    ssxy = ((x - xm) * (y - ym)).sum()
    slope     = ssxy / ssxx
    intercept = ym - slope * xm
    r         = np.clip(ssxy / np.sqrt(ssxx * ssyy + 1e-300), -1, 1)
    ss_res    = ssyy - slope * ssxy
    se        = np.sqrt(max(ss_res, 0) / max((n - 2) * ssxx, 1e-300))
    t         = r * np.sqrt(n - 2) / np.sqrt(max(1 - r**2, 1e-12))
    # Simple t→p approximation (good enough for display)
    p = 2 * (1 - _norm_cdf(abs(t)))
    return slope, intercept, r, p, se


def _norm_cdf(z: float) -> float:
    """Standard-normal CDF via Horner approximation (Abramowitz & Stegun)."""
    z = float(z)
    if z < 0:
        return 1.0 - _norm_cdf(-z)
    t = 1 / (1 + 0.2316419 * z)
    poly = t * (0.319381530
                + t * (-0.356563782
                       + t * (1.781477937
                              + t * (-1.821255978
                                     + t * 1.330274429))))
    return 1.0 - (1 / math.sqrt(2 * math.pi)) * math.exp(-0.5 * z**2) * poly


def _acf(series: np.ndarray, max_lag: int) -> np.ndarray:
    """Autocorrelation function up to max_lag."""
    s   = np.asarray(series, float)
    s   = s[np.isfinite(s)]
    s   -= s.mean()
    var = (s**2).mean()
    if var < 1e-12:
        return np.zeros(max_lag + 1)
    return np.array([(s[:len(s)-k] * s[k:]).mean() / var for k in range(max_lag + 1)])


def _hp_filter(y: np.ndarray, lam: float = 6.25) -> tuple:
    """
    Hodrick-Prescott filter (numpy only).
    Returns (trend, cycle).  lam=6.25 is standard for annual data.
    """
    y = np.asarray(y, float)
    n = len(y)
    # Build second-difference matrix D2
    D2 = np.zeros((n - 2, n))
    for i in range(n - 2):
        D2[i, i]   =  1
        D2[i, i+1] = -2
        D2[i, i+2] =  1
    A     = np.eye(n) + lam * D2.T @ D2
    trend = np.linalg.solve(A, y)
    return trend, y - trend


def _holt_winters(y: np.ndarray, alpha: float, beta: float, gamma: float,
                  season: int, n_ahead: int) -> tuple:
    """
    Triple (Holt-Winters) exponential smoothing — additive seasonality.
    Returns (fitted, forecast).
    alpha : level smoothing     (0 < alpha < 1)
    beta  : trend smoothing     (0 < beta  < 1)
    gamma : seasonal smoothing  (0 < gamma < 1)
    season: seasonal period (use 1 for non-seasonal / pure Holt)
    """
    y = np.asarray(y, float)
    n = len(y)

    if season <= 1 or n < 2 * season:
        # Holt's double exponential (no seasonality)
        level = y[0]
        trend = y[1] - y[0] if n > 1 else 0.0
        fitted = []
        for t in range(n):
            fitted.append(level + trend)
            prev_level = level
            if t < n:
                level = alpha * y[t] + (1 - alpha) * (level + trend)
                trend = beta  * (level - prev_level) + (1 - beta) * trend
        forecast = [level + (k + 1) * trend for k in range(n_ahead)]
        return np.array(fitted), np.array(forecast)

    # Initialise
    season_avgs = [np.mean(y[i * season:(i + 1) * season]) for i in range(n // season)]
    level  = season_avgs[0]
    trend  = (season_avgs[1] - season_avgs[0]) / season if len(season_avgs) > 1 else 0.0
    season_idx = list(range(season))
    seasonals  = [y[i] - level for i in range(season)]

    fitted = []
    for t in range(n):
        s_idx = t % season
        prev_level = level
        pred = level + trend + seasonals[s_idx]
        fitted.append(pred)
        level     = alpha * (y[t] - seasonals[s_idx]) + (1 - alpha) * (level + trend)
        trend     = beta  * (level - prev_level)       + (1 - beta)  * trend
        seasonals[s_idx] = gamma * (y[t] - level) + (1 - gamma) * seasonals[s_idx]

    forecast = []
    for k in range(n_ahead):
        s_idx = (n + k) % season
        forecast.append(level + (k + 1) * trend + seasonals[s_idx])

    return np.array(fitted), np.array(forecast)


def _moving_average(y: np.ndarray, window: int) -> np.ndarray:
    """Centred moving average; edges filled with NaN."""
    y   = np.asarray(y, float)
    out = np.full_like(y, np.nan)
    hw  = window // 2
    for i in range(hw, len(y) - hw):
        out[i] = y[i - hw : i + hw + 1].mean()
    return out


def _adf_stat(y: np.ndarray) -> tuple:
    """
    Augmented Dickey-Fuller test statistic (lag=1, numpy only).
    Returns (adf_stat, approx_p_label) where label is one of
    '<0.01', '<0.05', '<0.10', '>0.10' based on MacKinnon (1994) critical
    values for n≈25-35 with constant.
    """
    y  = np.asarray(y, float)
    y  = y[np.isfinite(y)]
    dy = np.diff(y)
    y_lag  = y[:-1]
    # include one lagged difference
    if len(dy) < 4:
        return np.nan, "n/a"
    dy1    = dy[:-1]
    dy_dep = dy[1:]
    y_lag  = y_lag[1:]
    X = np.column_stack([np.ones(len(y_lag)), y_lag, dy1])
    b = np.linalg.lstsq(X, dy_dep, rcond=None)[0]
    resid  = dy_dep - X @ b
    sigma2 = (resid**2).sum() / max(len(resid) - X.shape[1], 1)
    XtX_inv = np.linalg.pinv(X.T @ X)
    se_b1   = np.sqrt(sigma2 * XtX_inv[1, 1])
    adf     = b[1] / (se_b1 + 1e-300)
    # MacKinnon approximate critical values (constant, n≈30)
    if adf < -3.75:  return adf, "<0.01"
    if adf < -3.00:  return adf, "<0.05"
    if adf < -2.63:  return adf, "<0.10"
    return adf, ">0.10"


def _kmeans(X: np.ndarray, k: int, max_iter: int = 300, seed: int = 42) -> np.ndarray:
    """
    K-means clustering — pure numpy, k-means++ initialisation.
    X : (n_samples, n_features)
    Returns labels array of shape (n_samples,).
    """
    rng = np.random.default_rng(seed)
    n   = len(X)
    # k-means++ init
    centres = [X[rng.integers(n)]]
    for _ in range(k - 1):
        dists = np.array([min(np.sum((x - c)**2) for c in centres) for x in X])
        probs = dists / dists.sum()
        centres.append(X[rng.choice(n, p=probs)])
    centres = np.array(centres, float)

    labels = np.zeros(n, int)
    for _ in range(max_iter):
        # Assign
        dists  = np.array([[np.sum((x - c)**2) for c in centres] for x in X])
        new_lb = np.argmin(dists, axis=1)
        if np.all(new_lb == labels):
            break
        labels = new_lb
        # Update
        for j in range(k):
            mask = labels == j
            if mask.any():
                centres[j] = X[mask].mean(axis=0)
    return labels



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\ProgramData\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\ProgramData\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
# =============================================================================
# LOAD & PREP
# =============================================================================
print("\n" + "="*70)
print("SECTION 1 — LOAD & PREP")
print("="*70)

raw = pd.read_csv(RAW_PATH)
raw.columns = raw.columns.str.strip().str.upper()
raw.drop(columns=["PRIMARY_KEY"], inplace=True)

NON_STATES = {"DODEA", "NATIONAL"}
df = raw[~raw["STATE"].str.upper().isin(NON_STATES)].copy()

NUM_COLS = [
    "ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE", "STATE_REVENUE", "LOCAL_REVENUE",
    "TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
    "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE",
    "GRADES_4_G", "GRADES_8_G", "GRADES_ALL_G",
    "AVG_MATH_4_SCORE", "AVG_MATH_8_SCORE",
    "AVG_READING_4_SCORE", "AVG_READING_8_SCORE",
]
for col in NUM_COLS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df["STATE"] = df["STATE"].str.strip().str.upper().astype("string")
df["YEAR"]  = df["YEAR"].astype(int)
df.drop_duplicates(inplace=True)
df.sort_values(["STATE", "YEAR"], inplace=True)
df.reset_index(drop=True, inplace=True)

# Per-pupil features
enroll    = df["ENROLL"].replace(0, np.nan)
total_exp = df["TOTAL_EXPENDITURE"].replace(0, np.nan)
total_rev = df["TOTAL_REVENUE"].replace(0, np.nan)

df["EXPENDITURE_PER_PUPIL"] = df["TOTAL_EXPENDITURE"] / enroll
df["REVENUE_PER_PUPIL"]     = df["TOTAL_REVENUE"]     / enroll
df["PCT_FEDERAL"] = df["FEDERAL_REVENUE"] / total_rev * 100
df["PCT_STATE"]   = df["STATE_REVENUE"]   / total_rev * 100
df["PCT_LOCAL"]   = df["LOCAL_REVENUE"]   / total_rev * 100

df["AVG_COMPOSITE_SCORE"] = df[[
    "AVG_MATH_4_SCORE","AVG_MATH_8_SCORE",
    "AVG_READING_4_SCORE","AVG_READING_8_SCORE"
]].mean(axis=1, skipna=True)
df["AVG_MATH_SCORE"]    = df[["AVG_MATH_4_SCORE",    "AVG_MATH_8_SCORE"]].mean(axis=1, skipna=True)
df["AVG_READING_SCORE"] = df[["AVG_READING_4_SCORE", "AVG_READING_8_SCORE"]].mean(axis=1, skipna=True)

# Spotlight subset
spot = {s: df[df["STATE"] == s].sort_values("YEAR").reset_index(drop=True)
        for s in SPOTLIGHT_STATES}

METRIC   = "EXPENDITURE_PER_PUPIL"
SCORE    = "AVG_COMPOSITE_SCORE"
ALL_YMIN = min(df["YEAR"].min(), 1992)

print(f"Loaded {df.shape[0]:,} rows — {df['STATE'].nunique()} states")
print("Spotlight states:", ", ".join(SPOTLIGHT_STATES))


SECTION 1 — LOAD & PREP
Loaded 1,683 rows — 51 states
Spotlight states: PENNSYLVANIA, ALABAMA, CALIFORNIA, NEBRASKA


In [5]:
# =============================================================================
# STATIONARITY & TREND DETECTION
# =============================================================================
print("\n" + "="*70)
print("SECTION 2 — STATIONARITY & TREND DETECTION")
print("="*70)

# ── FIG 2-A  |  ADF test results — all states ─────────────────────────────
print("\n── Fig 2-A  ADF stationarity test — all states")

adf_results = {}
for state, grp in df.groupby("STATE"):
    series = grp.sort_values("YEAR")[METRIC].dropna().values
    stat, plabel = _adf_stat(series)
    adf_results[state] = {"stat": stat, "p_label": plabel}

adf_df   = pd.DataFrame(adf_results).T.dropna()
adf_df["stat"] = adf_df["stat"].astype(float)
adf_df   = adf_df.sort_values("stat")

p_colors = {
    "<0.01" : GREEN,
    "<0.05" : YELLOW,
    "<0.10" : ORANGE,
    ">0.10" : RED,
}
bar_colors = [p_colors.get(str(p), BLACK) for p in adf_df["p_label"]]

fig, ax = plt.subplots(figsize=(10, 16))
ax.barh(adf_df.index.str.replace("_"," ").str.title(),
        adf_df["stat"].values, color=bar_colors, edgecolor="white")
ax.axvline(-3.75, color=GREEN,  linestyle="--", linewidth=1.2, label="p<0.01  (−3.75)")
ax.axvline(-3.00, color=YELLOW, linestyle="--", linewidth=1.2, label="p<0.05  (−3.00)")
ax.axvline(-2.63, color=ORANGE, linestyle="--", linewidth=1.2, label="p<0.10  (−2.63)")
ax.set_xlabel("ADF Test Statistic  (more negative → stronger rejection of unit root)")
ax.set_title("ADF Stationarity Test — Per-Pupil Expenditure\n"
             "Bars in red indicate non-stationary series (likely trend-driven)")
ax.legend(loc="lower right")
fig.tight_layout()
save(fig, "02a_adf_stationarity.png")

stationary_n = sum(1 for r in adf_results.values() if r["p_label"] in ("<0.01","<0.05"))
print(f"   {stationary_n} / {len(adf_results)} states reject unit root at p<0.05")

# ── FIG 2-B  |  Spotlight — raw series with linear trend overlaid ─────────
print("── Fig 2-B  Spotlight states — raw series + linear trend")

fig, axes = plt.subplots(2, 2, figsize=(14, 9), sharex=False)
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub  = spot[state].dropna(subset=[METRIC])
    yrs  = sub["YEAR"].values.astype(float)
    vals = sub[METRIC].values

    slope, intercept, r, p, _ = _ols(yrs, vals)
    trend_line = intercept + slope * yrs

    ax.fill_between(yrs, vals, alpha=0.18, color=color)
    ax.plot(yrs, vals, linewidth=2.2, color=color, marker="o", markersize=4, label="Actual")
    ax.plot(yrs, trend_line, linewidth=1.8, color=BLACK, linestyle="--",
            label=f"Linear trend  r²={r**2:.2f}")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_title(label, color=color, fontweight="bold")
    ax.set_xlabel("Year")
    ax.set_ylabel("Exp / Pupil")
    ax.legend(fontsize=8)

fig.suptitle("Per-Pupil Expenditure — Raw Series & Linear Trend", fontsize=15, y=1.01)
fig.tight_layout()
save(fig, "02b_spotlight_linear_trend.png")


SECTION 2 — STATIONARITY & TREND DETECTION

── Fig 2-A  ADF stationarity test — all states
  ✓  Saved  outputs/figures/ts/02a_adf_stationarity.png
   0 / 51 states reject unit root at p<0.05
── Fig 2-B  Spotlight states — raw series + linear trend
  ✓  Saved  outputs/figures/ts/02b_spotlight_linear_trend.png


In [7]:
# =============================================================================
# TREND DECOMPOSITION  (Hodrick-Prescott)
# =============================================================================
print("\n" + "="*70)
print("SECTION 3 — HP FILTER DECOMPOSITION")
print("="*70)

print("\n── Fig 3-A  HP decomposition — national median")

nat_med = df.groupby("YEAR")[METRIC].median().dropna()
hp_trend, hp_cycle = _hp_filter(nat_med.values, lam=6.25)

fig, axes = plt.subplots(3, 1, figsize=(13, 10), sharex=True)

axes[0].plot(nat_med.index, nat_med.values, color=BLUE, linewidth=2.2, label="Observed")
axes[0].plot(nat_med.index, hp_trend,       color=RED,  linewidth=2.0, linestyle="--", label="HP Trend")
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
axes[0].set_title("Observed + HP Trend")
axes[0].legend()

axes[1].fill_between(nat_med.index, hp_cycle, alpha=0.45, color=ORANGE)
axes[1].plot(nat_med.index, hp_cycle, color=ORANGE, linewidth=1.8)
axes[1].axhline(0, color=BLACK, linewidth=1, linestyle=":")
axes[1].yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
axes[1].set_title("Cyclical Component  (deviations from trend)")

residuals = nat_med.values - hp_trend
axes[2].bar(nat_med.index, residuals, color=PURPLE, alpha=0.7, width=0.8)
axes[2].axhline(0, color=BLACK, linewidth=1, linestyle=":")
axes[2].yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
axes[2].set_title("Residuals (Observed − Trend)")
axes[2].set_xlabel("Year")

fig.suptitle("Hodrick-Prescott Decomposition — National Median Per-Pupil Expenditure",
             fontsize=14, y=1.01)
fig.tight_layout()
save(fig, "03a_hp_decomposition_national.png")

# ── FIG 3-B  |  HP cycle comparison across four spotlight states ──────────
print("── Fig 3-B  HP cycle — spotlight states")

fig, ax = plt.subplots(figsize=(13, 6))
for state, label, color, marker in zip(SPOTLIGHT_STATES, SPOTLIGHT_LABELS,
                                        SPOTLIGHT_COLORS, SPOTLIGHT_MARKERS):
    sub = spot[state].dropna(subset=[METRIC])
    if len(sub) < 6:
        continue
    tr, cy = _hp_filter(sub[METRIC].values, lam=6.25)
    ax.plot(sub["YEAR"].values, cy, linewidth=2.0, color=color,
            marker=marker, markersize=4, label=label)

ax.axhline(0, color=BLACK, linewidth=1.2, linestyle="--")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax.set_xlabel("Year")
ax.set_ylabel("Cyclical Deviation from HP Trend ($)")
ax.set_title("HP Cyclical Component — Spotlight States\n"
             "Positive = spending above its own trend; Negative = below")
ax.legend()
fig.tight_layout()
save(fig, "03b_hp_cycle_spotlight.png")


SECTION 3 — HP FILTER DECOMPOSITION

── Fig 3-A  HP decomposition — national median
  ✓  Saved  outputs/figures/ts/03a_hp_decomposition_national.png
── Fig 3-B  HP cycle — spotlight states
  ✓  Saved  outputs/figures/ts/03b_hp_cycle_spotlight.png


In [9]:
# =============================================================================
# AUTOCORRELATION ANALYSIS
# =============================================================================
print("\n" + "="*70)
print("SECTION 4 — AUTOCORRELATION (ACF)")
print("="*70)

print("\n── Fig 4-A  ACF — spotlight states per-pupil expenditure")

MAX_LAG = 10
ci_band = 1.96 / np.sqrt(25)   # ~95% confidence band for n≈25

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub  = spot[state].dropna(subset=[METRIC])[METRIC].values
    acf  = _acf(sub, MAX_LAG)
    lags = np.arange(MAX_LAG + 1)
    ax.bar(lags, acf, color=color, alpha=0.8, edgecolor="white", width=0.6)
    ax.axhline( ci_band, color=BLACK, linestyle="--", linewidth=1, alpha=0.6, label="95% CI")
    ax.axhline(-ci_band, color=BLACK, linestyle="--", linewidth=1, alpha=0.6)
    ax.axhline(0, color=BLACK, linewidth=0.8)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel("Lag (years)")
    ax.set_ylabel("ACF")
    ax.set_title(label, color=color, fontweight="bold")
    ax.legend(fontsize=8)

fig.suptitle("Autocorrelation Function — Per-Pupil Expenditure\n"
             "High persistence (bars above CI at many lags) confirms a trending series",
             fontsize=13, y=1.02)
fig.tight_layout()
save(fig, "04a_acf_spotlight.png")

# ── FIG 4-B  |  ACF of first-differenced series ───────────────────────────
print("── Fig 4-B  ACF of first-differenced series (removes trend)")

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub  = spot[state].dropna(subset=[METRIC])[METRIC].values
    diff = np.diff(sub)
    acf  = _acf(diff, MAX_LAG)
    lags = np.arange(MAX_LAG + 1)
    ax.bar(lags, acf, color=color, alpha=0.8, edgecolor="white", width=0.6)
    ax.axhline( ci_band, color=BLACK, linestyle="--", linewidth=1, alpha=0.6)
    ax.axhline(-ci_band, color=BLACK, linestyle="--", linewidth=1, alpha=0.6)
    ax.axhline(0, color=BLACK, linewidth=0.8)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel("Lag (years)")
    ax.set_ylabel("ACF")
    ax.set_title(f"{label}  [Δ first-diff]", color=color, fontweight="bold")

fig.suptitle("ACF of First-Differenced Per-Pupil Expenditure\n"
             "Differencing removes the trend — most autocorrelation should now vanish",
             fontsize=13, y=1.02)
fig.tight_layout()
save(fig, "04b_acf_differenced.png")


# =============================================================================
# CROSS-CORRELATION: SPENDING → SCORES  (LAG ANALYSIS)
# =============================================================================
print("\n" + "="*70)
print("SECTION 5 — CROSS-CORRELATION: SPENDING → TEST SCORES")
print("="*70)

print("\n── Fig 5-A  Lag correlation — national level")

nat = df.groupby("YEAR")[[METRIC, SCORE]].median().dropna()
MAX_CROSS_LAG = 8
lag_r, lag_p = [], []

for lag in range(0, MAX_CROSS_LAG + 1):
    x = nat[METRIC].values[:len(nat) - lag]
    y = nat[SCORE].values[lag:]
    ok = np.isfinite(x) & np.isfinite(y)
    if ok.sum() < 5:
        lag_r.append(np.nan); lag_p.append(np.nan); continue
    r = np.corrcoef(x[ok], y[ok])[0, 1]
    _, _, _, p, _ = _ols(x[ok], y[ok])
    lag_r.append(r); lag_p.append(p)

lags = np.arange(MAX_CROSS_LAG + 1)
bar_c = [GREEN if p < 0.05 else ORANGE if p < 0.10 else RED
         for p in lag_p]

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(lags, lag_r, color=bar_c, edgecolor="white", width=0.6)
ax.axhline(0, color=BLACK, linewidth=0.8)
for lag, r, p in zip(lags, lag_r, lag_p):
    if np.isfinite(r):
        ax.text(lag, r + 0.01 * np.sign(r + 1e-9), f"p={p:.2f}",
                ha="center", va="bottom", fontsize=7.5, color=BLACK)
ax.set_xlabel("Lag (years) — spending at time t  vs.  score at t+lag")
ax.set_ylabel("Pearson r")
ax.set_title("Cross-Correlation: Per-Pupil Spending → Composite NAEP Score\n"
             "Green = p<0.05  |  Orange = p<0.10  |  Red = not significant")
ax.set_xticks(lags)
fig.tight_layout()
save(fig, "05a_cross_correlation_lag.png")

# ── FIG 5-B  |  Lag scatter for strongest lag ─────────────────────────────
print("── Fig 5-B  Scatter at best lag")

best_lag = int(np.nanargmax(np.abs(lag_r)))
print(f"   Best lag = {best_lag} yr  (r={lag_r[best_lag]:.3f})")

x_bl = nat[METRIC].values[:len(nat) - best_lag]
y_bl = nat[SCORE].values[best_lag:]
ok   = np.isfinite(x_bl) & np.isfinite(y_bl)
m, b, r_bl, _, _ = _ols(x_bl[ok], y_bl[ok])

fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(x_bl[ok], y_bl[ok], color=BLUE, s=60, edgecolors="white", zorder=3)
x_line = np.linspace(x_bl[ok].min(), x_bl[ok].max(), 100)
ax.plot(x_line, m * x_line + b, color=RED, linewidth=2,
        label=f"OLS  r={r_bl:.2f}  lag={best_lag}yr")
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel(f"Per-Pupil Expenditure (year t)")
ax.set_ylabel(f"Composite NAEP Score (year t+{best_lag})")
ax.set_title(f"Spending at t  vs.  Score at t+{best_lag}\n(national medians)")
ax.legend()
fig.tight_layout()
save(fig, "05b_lag_scatter_best.png")


SECTION 4 — AUTOCORRELATION (ACF)

── Fig 4-A  ACF — spotlight states per-pupil expenditure
  ✓  Saved  outputs/figures/ts/04a_acf_spotlight.png
── Fig 4-B  ACF of first-differenced series (removes trend)
  ✓  Saved  outputs/figures/ts/04b_acf_differenced.png

SECTION 5 — CROSS-CORRELATION: SPENDING → TEST SCORES

── Fig 5-A  Lag correlation — national level
  ✓  Saved  outputs/figures/ts/05a_cross_correlation_lag.png
── Fig 5-B  Scatter at best lag
   Best lag = 4 yr  (r=0.859)
  ✓  Saved  outputs/figures/ts/05b_lag_scatter_best.png


In [11]:
# =============================================================================
# STRUCTURAL BREAK DETECTION
# =============================================================================
print("\n" + "="*70)
print("SECTION 6 — STRUCTURAL BREAK DETECTION")
print("="*70)

print("\n── Fig 6-A  Rolling variance — structural breaks")

# Use national median; rolling 5-yr variance in expenditure growth
nat_exp = df.groupby("YEAR")[METRIC].median().dropna()
exp_growth = nat_exp.pct_change().dropna() * 100
years_g    = exp_growth.index.values

WINDOW = 5
roll_var = pd.Series(exp_growth.values, index=years_g).rolling(WINDOW, min_periods=3).var()

# Chow-style: find year where mean before vs. after differs most
best_break, best_stat = None, -np.inf
for i in range(4, len(exp_growth) - 4):
    before = exp_growth.values[:i]
    after  = exp_growth.values[i:]
    stat   = abs(before.mean() - after.mean()) / np.sqrt(
        before.var() / len(before) + after.var() / len(after) + 1e-9)
    if stat > best_stat:
        best_stat, best_break = stat, years_g[i]

print(f"   Most likely structural break year: {best_break}")

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(13, 8), sharex=True)
ax1.plot(nat_exp.index, nat_exp.values, color=BLUE, linewidth=2.2, marker="o", markersize=3)
ax1.axvline(best_break, color=RED, linewidth=2, linestyle="--",
            label=f"Break: {best_break}")
ax1.axvline(2008, color=ORANGE, linewidth=1.5, linestyle=":", label="2008 (GFC)")
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax1.set_ylabel("Median Exp / Pupil")
ax1.set_title("National Median Per-Pupil Expenditure")
ax1.legend()

ax2.fill_between(roll_var.index, roll_var.values, alpha=0.4, color=PURPLE)
ax2.plot(roll_var.index, roll_var.values, color=PURPLE, linewidth=1.8)
ax2.axvline(best_break, color=RED, linewidth=2, linestyle="--")
ax2.axvline(2008, color=ORANGE, linewidth=1.5, linestyle=":")
ax2.set_ylabel(f"{WINDOW}-yr Rolling Variance of Growth Rate")
ax2.set_xlabel("Year")
ax2.set_title("Rolling Variance (elevated variance signals a regime change)")

fig.suptitle("Structural Break Detection in Spending Growth", fontsize=14, y=1.01)
fig.tight_layout()
save(fig, "06a_structural_break.png")

# ── FIG 6-B  |  Spotlight — break detection ───────────────────────────────
print("── Fig 6-B  Structural breaks — spotlight states")

fig, axes = plt.subplots(2, 2, figsize=(14, 9), sharex=False)
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub = spot[state].dropna(subset=[METRIC]).sort_values("YEAR")
    eg  = sub[METRIC].pct_change().dropna() * 100
    yg  = sub["YEAR"].values[1:]

    sb, ss = None, -np.inf
    for i in range(3, len(eg) - 3):
        bef = eg.values[:i]; aft = eg.values[i:]
        st  = abs(bef.mean() - aft.mean()) / np.sqrt(
            bef.var()/len(bef) + aft.var()/len(aft) + 1e-9)
        if st > ss:
            ss, sb = st, yg[i]

    ax.fill_between(sub["YEAR"], sub[METRIC], alpha=0.15, color=color)
    ax.plot(sub["YEAR"], sub[METRIC], color=color, linewidth=2.2,
            marker="o", markersize=3)
    if sb:
        ax.axvline(sb, color=BLACK, linewidth=2, linestyle="--",
                   label=f"Break ≈ {sb}")
    ax.axvline(2008, color=ORANGE, linewidth=1.5, linestyle=":", label="2008 GFC")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_title(label, color=color, fontweight="bold")
    ax.set_xlabel("Year")
    ax.legend(fontsize=8)

fig.suptitle("Structural Break Detection — Spotlight States", fontsize=14, y=1.01)
fig.tight_layout()
save(fig, "06b_structural_break_spotlight.png")


SECTION 6 — STRUCTURAL BREAK DETECTION

── Fig 6-A  Rolling variance — structural breaks
   Most likely structural break year: 2010
  ✓  Saved  outputs/figures/ts/06a_structural_break.png
── Fig 6-B  Structural breaks — spotlight states
  ✓  Saved  outputs/figures/ts/06b_structural_break_spotlight.png


In [13]:
# =============================================================================
# MODEL COMPARISON — SPOTLIGHT STATES
# Models:  (A) Naïve / last-value  (B) Moving Average  (C) Linear Trend
#          (D) Holt double exponential  (best alpha/beta via grid search)
# =============================================================================
print("\n" + "="*70)
print("SECTION 7 — BASIC TIME-SERIES MODEL COMPARISON")
print("="*70)

N_AHEAD   = 5
HOLD_OUT  = 5          # last N years used as test set

def _rmse(a, b):
    ok = np.isfinite(a) & np.isfinite(b)
    return np.sqrt(((a[ok] - b[ok])**2).mean()) if ok.sum() else np.nan


print("\n── Fig 7-A/B/C/D  Model comparison per spotlight state")

summary_rows = []

for state, label, color in zip(SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub  = spot[state].dropna(subset=[METRIC]).sort_values("YEAR")
    yrs  = sub["YEAR"].values
    vals = sub[METRIC].values
    n    = len(vals)
    if n < HOLD_OUT + 4:
        continue

    train_y = vals[:-HOLD_OUT]
    test_y  = vals[-HOLD_OUT:]
    train_x = yrs[:-HOLD_OUT]
    test_x  = yrs[-HOLD_OUT:]

    # (A) Naïve
    naive_fit  = np.full(len(train_y), np.nan)
    naive_pred = np.full(HOLD_OUT, train_y[-1])

    # (B) Moving average (window=3)
    ma_fit  = _moving_average(train_y, 3)
    ma_pred = np.full(HOLD_OUT, np.nanmean(train_y[-3:]))

    # (C) Linear trend
    sl, ic, r_lt, _, _ = _ols(train_x, train_y)
    lin_fit  = ic + sl * train_x
    lin_pred = ic + sl * test_x

    # (D) Holt double exponential — grid search alpha/beta
    best_rmse, best_alpha, best_beta = np.inf, 0.3, 0.1
    for a in np.arange(0.1, 0.9, 0.1):
        for b in np.arange(0.05, 0.5, 0.05):
            try:
                f, _ = _holt_winters(train_y, alpha=a, beta=b, gamma=0, season=1, n_ahead=0)
                err  = _rmse(train_y, f)
                if err < best_rmse:
                    best_rmse, best_alpha, best_beta = err, a, b
            except Exception:
                continue
    hw_fit, hw_fc = _holt_winters(train_y, best_alpha, best_beta, 0, 1, HOLD_OUT)

    rmse_naive = _rmse(test_y, naive_pred)
    rmse_ma    = _rmse(test_y, ma_pred)
    rmse_lin   = _rmse(test_y, lin_pred)
    rmse_hw    = _rmse(test_y, hw_fc)

    summary_rows.append({
        "State": label, "Naïve": rmse_naive, "MA(3)": rmse_ma,
        "Linear": rmse_lin, f"Holt(α={best_alpha:.1f},β={best_beta:.2f})": rmse_hw,
    })

    # ── one figure per state (4 panels: fit, residuals, forecast, score overlay) ──
    fig, axes = plt.subplots(2, 2, figsize=(14, 9))
    fig.suptitle(f"Model Comparison — {label}", fontsize=15,
                 color=color, fontweight="bold")

    # Panel 1 — fits on training set
    ax = axes[0, 0]
    ax.plot(train_x, train_y, color=BLACK, linewidth=2, label="Actual (train)", zorder=5)
    ax.plot(train_x, lin_fit, color=GREEN,  linewidth=1.8, linestyle="--", label="Linear")
    ax.plot(train_x, ma_fit,  color=ORANGE, linewidth=1.8, linestyle="-.", label="MA(3)")
    ax.plot(train_x, hw_fit,  color=PURPLE, linewidth=1.8, linestyle=":",  label="Holt")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_title("In-sample Fit (training)")
    ax.legend(fontsize=8)

    # Panel 2 — forecast vs. actual
    ax = axes[0, 1]
    ax.plot(yrs,     vals,      color=BLACK,   linewidth=2,   label="Actual (all)", zorder=5)
    ax.plot(test_x,  naive_pred, color=YELLOW,  linewidth=1.8, linestyle="--", label=f"Naïve  RMSE={rmse_naive:,.0f}")
    ax.plot(test_x,  lin_pred,   color=GREEN,   linewidth=1.8, linestyle="--", label=f"Linear RMSE={rmse_lin:,.0f}")
    ax.plot(test_x,  ma_pred,    color=ORANGE,  linewidth=1.8, linestyle="-.", label=f"MA(3)  RMSE={rmse_ma:,.0f}")
    ax.plot(test_x,  hw_fc,      color=PURPLE,  linewidth=1.8, linestyle=":",  label=f"Holt   RMSE={rmse_hw:,.0f}")
    ax.axvline(test_x[0] - 0.5, color=RED, linewidth=1.5, linestyle=":", alpha=0.7)
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_title("Out-of-sample Forecast (last 5 years as test)")
    ax.legend(fontsize=7.5)

    # Panel 3 — residuals from best model (Holt)
    ax = axes[1, 0]
    hw_resid = train_y - hw_fit
    ax.bar(train_x, hw_resid, color=color, alpha=0.75, edgecolor="white")
    ax.axhline(0, color=BLACK, linewidth=1)
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
    ax.set_title(f"Holt Residuals (train)")

    # Panel 4 — RMSE bar chart
    ax = axes[1, 1]
    model_names  = ["Naïve", "MA(3)", "Linear", "Holt"]
    model_rmses  = [rmse_naive, rmse_ma, rmse_lin, rmse_hw]
    model_colors = [YELLOW, ORANGE, GREEN, PURPLE]
    bars = ax.bar(model_names, model_rmses, color=model_colors, edgecolor="white", width=0.5)
    for bar, val in zip(bars, model_rmses):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(model_rmses)*0.01,
                f"${val:,.0f}", ha="center", va="bottom", fontsize=9)
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
    ax.set_title("Test RMSE by Model  (lower = better)")
    ax.set_ylabel("RMSE ($)")

    fig.tight_layout()
    fname = f"07_{state.lower()[:4]}_model_comparison.png"
    save(fig, fname)

# ── FIG 7-E  |  Summary RMSE table as heatmap ─────────────────────────────
print("── Fig 7-E  RMSE summary heatmap")

if summary_rows:
    rmse_df = pd.DataFrame(summary_rows).set_index("State")
    rmse_df.columns = [c.split("(")[0].strip() for c in rmse_df.columns]

    data    = rmse_df.values.astype(float)
    r_norm  = mcolors.Normalize(vmin=np.nanmin(data), vmax=np.nanmax(data))
    cmap_rm = mcm.RdYlGn_r

    fig, ax = plt.subplots(figsize=(10, 5))
    im = ax.imshow(data, cmap=cmap_rm, norm=r_norm, aspect="auto")
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            ax.text(j, i, f"${data[i,j]:,.0f}", ha="center", va="center",
                    fontsize=9, color="white" if data[i,j] > np.nanmedian(data) else BLACK)
    ax.set_xticks(range(data.shape[1]))
    ax.set_xticklabels(rmse_df.columns, fontsize=10)
    ax.set_yticks(range(data.shape[0]))
    ax.set_yticklabels(rmse_df.index, fontsize=10)
    fig.colorbar(im, ax=ax, fraction=0.025, label="RMSE ($)")
    ax.set_title("Test-Set RMSE by Model & State  (green = better)")
    fig.tight_layout()
    save(fig, "07e_rmse_summary_heatmap.png")




SECTION 7 — BASIC TIME-SERIES MODEL COMPARISON

── Fig 7-A/B/C/D  Model comparison per spotlight state
  ✓  Saved  outputs/figures/ts/07_penn_model_comparison.png
  ✓  Saved  outputs/figures/ts/07_alab_model_comparison.png
  ✓  Saved  outputs/figures/ts/07_cali_model_comparison.png
  ✓  Saved  outputs/figures/ts/07_nebr_model_comparison.png
── Fig 7-E  RMSE summary heatmap
  ✓  Saved  outputs/figures/ts/07e_rmse_summary_heatmap.png


In [15]:
# =============================================================================
# ADVANCED MODEL — HOLT-WINTERS TRIPLE EXPONENTIAL SMOOTHING
# Applied to all four spotlight states with 10-year forward projection
# and bootstrap confidence bands (numpy only)
# =============================================================================
print("\n" + "="*70)
print("SECTION 8 — ADVANCED: HOLT-WINTERS + BOOTSTRAP FORECAST")
print("="*70)

N_AHEAD_LONG = 10
N_BOOT       = 300

print(f"\n── Fig 8-A  Holt-Winters 10-year forecast — all four states")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub  = spot[state].dropna(subset=[METRIC]).sort_values("YEAR")
    yrs  = sub["YEAR"].values
    vals = sub[METRIC].values
    n    = len(vals)
    if n < 8:
        continue

    # Grid-search best alpha/beta on full series
    best_rmse, best_a, best_b = np.inf, 0.3, 0.1
    for a in np.arange(0.05, 0.95, 0.05):
        for b in np.arange(0.01, 0.50, 0.05):
            try:
                f, _ = _holt_winters(vals, a, b, 0, 1, 0)
                err  = _rmse(vals, f)
                if err < best_rmse:
                    best_rmse, best_a, best_b = err, a, b
            except Exception:
                continue

    fitted, forecast = _holt_winters(vals, best_a, best_b, 0, 1, N_AHEAD_LONG)
    resid  = vals - fitted

    # Bootstrap confidence bands: resample residuals N_BOOT times
    fc_boot = []
    rng     = np.random.default_rng(0)
    for _ in range(N_BOOT):
        noise = rng.choice(resid, size=N_AHEAD_LONG, replace=True)
        fc_boot.append(forecast + noise)
    fc_boot  = np.array(fc_boot)
    fc_lo    = np.percentile(fc_boot, 10, axis=0)
    fc_hi    = np.percentile(fc_boot, 90, axis=0)
    fc_yrs   = np.arange(yrs[-1] + 1, yrs[-1] + N_AHEAD_LONG + 1)

    ax.fill_between(fc_yrs, fc_lo, fc_hi, alpha=0.22, color=color,
                    label="80% bootstrap CI")
    ax.plot(yrs,    vals,     color=color,  linewidth=2.2, marker="o", markersize=3,
            label="Actual")
    ax.plot(yrs,    fitted,   color=BLACK,  linewidth=1.6, linestyle="--",
            label=f"HW fit  (α={best_a:.2f}, β={best_b:.2f})")
    ax.plot(fc_yrs, forecast, color=color,  linewidth=2.2, linestyle=":",
            marker="^", markersize=5, label="HW forecast")
    ax.axvline(yrs[-1] + 0.5, color=RED, linewidth=1.2, linestyle=":", alpha=0.6)

    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_title(label, color=color, fontweight="bold")
    ax.set_xlabel("Year")
    ax.set_ylabel("Exp / Pupil")
    ax.legend(fontsize=7.5)

fig.suptitle(
    "Holt-Winters Double Exponential Smoothing — 10-Year Forecast\n"
    "Shaded band = 80% bootstrap confidence interval (300 resamples of residuals)",
    fontsize=13, y=1.01,
)
fig.tight_layout()
save(fig, "08a_holt_winters_forecast.png")

# ── FIG 8-B  |  Forecast paths — all four on one chart ───────────────────
print("── Fig 8-B  Combined forecast paths")

fig, ax = plt.subplots(figsize=(14, 7))
for state, label, color, marker in zip(SPOTLIGHT_STATES, SPOTLIGHT_LABELS,
                                        SPOTLIGHT_COLORS, SPOTLIGHT_MARKERS):
    sub  = spot[state].dropna(subset=[METRIC]).sort_values("YEAR")
    yrs  = sub["YEAR"].values
    vals = sub[METRIC].values
    if len(vals) < 8:
        continue

    best_rmse, best_a, best_b = np.inf, 0.3, 0.1
    for a in np.arange(0.05, 0.95, 0.05):
        for b in np.arange(0.01, 0.50, 0.05):
            try:
                f, _ = _holt_winters(vals, a, b, 0, 1, 0)
                err  = _rmse(vals, f)
                if err < best_rmse:
                    best_rmse, best_a, best_b = err, a, b
            except Exception:
                continue

    _, forecast = _holt_winters(vals, best_a, best_b, 0, 1, N_AHEAD_LONG)
    fc_yrs = np.arange(yrs[-1] + 1, yrs[-1] + N_AHEAD_LONG + 1)

    ax.plot(yrs,    vals,     color=color, linewidth=2.2, marker=marker,
            markersize=4, label=label)
    ax.plot(fc_yrs, forecast, color=color, linewidth=2.0, linestyle=":",
            marker=marker, markersize=4)

ax.axvline(yrs[-1] + 0.5, color=BLACK, linewidth=1.5, linestyle="--",
           label="Forecast start →")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
ax.set_xlabel("Year")
ax.set_ylabel("Per-Pupil Expenditure")
ax.set_title("Holt-Winters Forecast — All Spotlight States on One Canvas\n"
             "Solid = observed  |  Dotted = projected")
ax.legend()
fig.tight_layout()
save(fig, "08b_forecast_combined.png")


SECTION 8 — ADVANCED: HOLT-WINTERS + BOOTSTRAP FORECAST

── Fig 8-A  Holt-Winters 10-year forecast — all four states
  ✓  Saved  outputs/figures/ts/08a_holt_winters_forecast.png
── Fig 8-B  Combined forecast paths
  ✓  Saved  outputs/figures/ts/08b_forecast_combined.png


In [17]:
# =============================================================================
# K-MEANS CLUSTERING OF STATE SPENDING TRAJECTORIES
# =============================================================================
print("\n" + "="*70)
print("SECTION 9 — K-MEANS TRAJECTORY CLUSTERING  (numpy k-means++)")
print("="*70)

# Build matrix: one row per state, columns = per-pupil exp normalised to z-score
CLUSTER_YEARS = list(range(1993, 2020))
pivot = (
    df[df["YEAR"].isin(CLUSTER_YEARS)]
    .pivot_table(index="STATE", columns="YEAR", values=METRIC)
    .dropna(thresh=15)          # require at least 15 years of data
)

# Drop fully-empty columns, interpolate gaps, then drop any still-NaN columns
pivot        = pivot.dropna(axis=1, how="all")
pivot_filled = pivot.interpolate(axis=1, limit_direction="both").dropna(axis=1)
CLUSTER_YEARS = list(pivot_filled.columns)   # sync to actual available years
row_mean     = pivot_filled.mean(axis=1).values[:, None]
row_std      = pivot_filled.std(axis=1).values[:, None] + 1e-9
X_norm       = (pivot_filled.values - row_mean) / row_std

K = 4
labels = _kmeans(X_norm, K, seed=42)
state_labels = dict(zip(pivot.index, labels))

# ── FIG 9-A  |  Cluster archetype trajectories ────────────────────────────
print("\n── Fig 9-A  Cluster archetypes")

clust_colors = [BLUE, RED, GREEN, ORANGE]

fig, ax = plt.subplots(figsize=(13, 6))
for k, ck in enumerate(clust_colors):
    mask     = labels == k
    members  = X_norm[mask]
    centroid = members.mean(axis=0)
    n_states = mask.sum()
    ax.fill_between(CLUSTER_YEARS, centroid - members.std(axis=0),
                    centroid + members.std(axis=0), alpha=0.15, color=ck)
    ax.plot(CLUSTER_YEARS, centroid, linewidth=2.5, color=ck,
            label=f"Cluster {k+1}  (n={n_states})")

ax.axhline(0, color=BLACK, linewidth=0.8, linestyle=":")
ax.set_xlabel("Year")
ax.set_ylabel("Z-scored Per-Pupil Expenditure")
ax.set_title("K-Means Cluster Archetypes — State Spending Trajectories\n"
             "Shaded band = ±1 std dev across cluster members")
ax.legend()
fig.tight_layout()
save(fig, "09a_cluster_archetypes.png")

# ── FIG 9-B  |  Which states are in which cluster ─────────────────────────
print("── Fig 9-B  Cluster membership table")

cluster_df = pd.DataFrame(
    {"STATE": pivot.index, "CLUSTER": labels + 1}
).sort_values(["CLUSTER", "STATE"])

fig, ax = plt.subplots(figsize=(12, 8))
ax.axis("off")
col_headers = [f"Cluster {k+1}" for k in range(K)]
col_data     = [
    cluster_df[cluster_df["CLUSTER"] == k+1]["STATE"]
    .str.replace("_", " ").str.title().tolist()
    for k in range(K)
]
max_len = max(len(c) for c in col_data)
col_data = [c + [""] * (max_len - len(c)) for c in col_data]

table_data = [[col_data[k][i] for k in range(K)] for i in range(max_len)]
tbl = ax.table(
    cellText=table_data,
    colLabels=col_headers,
    cellLoc="center", loc="center",
)
tbl.auto_set_font_size(False)
tbl.set_fontsize(9)
for k, ck in enumerate(clust_colors):
    tbl[(0, k)].set_facecolor(ck)
    tbl[(0, k)].set_text_props(color="white", fontweight="bold")
    for i in range(1, max_len + 1):
        tbl[(i, k)].set_facecolor("#FFFDE7" if i % 2 else "#FFF9C4")
ax.set_title("Cluster Membership by State", fontsize=13, pad=20)
fig.tight_layout()
save(fig, "09b_cluster_membership.png")

for k in range(K):
    members = cluster_df[cluster_df["CLUSTER"] == k+1]["STATE"].tolist()
    print(f"   Cluster {k+1}: {', '.join(members)}")

# ── FIG 9-C  |  Spotlight states highlighted in cluster context ───────────
print("── Fig 9-C  Spotlight states in cluster context")

fig, axes = plt.subplots(2, 2, figsize=(14, 9))
axes = axes.flatten()

for ax, state, slabel, scolor in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    if state not in state_labels:
        continue
    k        = state_labels[state]
    mask     = labels == k
    members  = X_norm[mask]
    centroid = members.mean(axis=0)

    ax.fill_between(CLUSTER_YEARS, centroid - members.std(axis=0),
                    centroid + members.std(axis=0), alpha=0.15, color=clust_colors[k])
    for m in members:
        ax.plot(CLUSTER_YEARS, m, linewidth=0.7, color=clust_colors[k], alpha=0.3)
    ax.plot(CLUSTER_YEARS, centroid, linewidth=1.8, color=clust_colors[k],
            linestyle="--", label=f"Cluster {k+1} centroid")

    if state in pivot.index:
        idx   = list(pivot.index).index(state)
        ax.plot(CLUSTER_YEARS, X_norm[idx], linewidth=2.8, color=scolor,
                marker="o", markersize=4, zorder=5, label=slabel)
    ax.axhline(0, color=BLACK, linewidth=0.8, linestyle=":")
    ax.set_title(f"{slabel} — in Cluster {k+1}", color=scolor, fontweight="bold")
    ax.set_xlabel("Year")
    ax.set_ylabel("Z-scored Exp / Pupil")
    ax.legend(fontsize=8)

fig.suptitle("Spotlight States Within Their Spending Trajectory Clusters",
             fontsize=14, y=1.01)
fig.tight_layout()
save(fig, "09c_spotlight_in_cluster.png")


SECTION 9 — K-MEANS TRAJECTORY CLUSTERING  (numpy k-means++)

── Fig 9-A  Cluster archetypes
  ✓  Saved  outputs/figures/ts/09a_cluster_archetypes.png
── Fig 9-B  Cluster membership table
  ✓  Saved  outputs/figures/ts/09b_cluster_membership.png
   Cluster 1: ARIZONA, FLORIDA, GEORGIA, IDAHO, NEVADA
   Cluster 2: ALASKA, ARKANSAS, CONNECTICUT, DISTRICT_OF_COLUMBIA, HAWAII, KANSAS, KENTUCKY, LOUISIANA, MARYLAND, MONTANA, NEBRASKA, NEW_HAMPSHIRE, NEW_JERSEY, NEW_YORK, NORTH_DAKOTA, PENNSYLVANIA, RHODE_ISLAND, WASHINGTON, WEST_VIRGINIA, WYOMING
   Cluster 3: DELAWARE, ILLINOIS, IOWA, MAINE, MASSACHUSETTS, MINNESOTA, MISSOURI, OHIO, OREGON, SOUTH_DAKOTA, TENNESSEE, VERMONT, WISCONSIN
   Cluster 4: ALABAMA, CALIFORNIA, COLORADO, INDIANA, MICHIGAN, MISSISSIPPI, NEW_MEXICO, NORTH_CAROLINA, OKLAHOMA, SOUTH_CAROLINA, TEXAS, UTAH, VIRGINIA
── Fig 9-C  Spotlight states in cluster context
  ✓  Saved  outputs/figures/ts/09c_spotlight_in_cluster.png


In [18]:
# =============================================================================
# HOW STATES TREND DIFFERENTLY
# =============================================================================
print("\n" + "="*70)
print("SECTION 10 — HOW STATES TREND DIFFERENTLY")
print("="*70)

# ── FIG 10-A  |  Annualised growth rate (1993-2019) distribution ──────────
print("\n── Fig 10-A  Annualised growth rate distribution")

growth_map = {}
for state, grp in df.groupby("STATE"):
    sub = grp.sort_values("YEAR").dropna(subset=[METRIC])
    if len(sub) < 10:
        continue
    first, last = sub[METRIC].iloc[0], sub[METRIC].iloc[-1]
    nyrs = sub["YEAR"].iloc[-1] - sub["YEAR"].iloc[0]
    if nyrs > 0 and first > 0:
        growth_map[state] = ((last / first) ** (1 / nyrs) - 1) * 100

growth_s = pd.Series(growth_map).sort_values()
n_states = len(growth_s)
bar_cols  = [BLUE if v >= growth_s.median() else RED for v in growth_s.values]

fig, ax = plt.subplots(figsize=(10, 15))
ax.barh(growth_s.index.str.replace("_", " ").str.title(),
        growth_s.values, color=bar_cols, edgecolor="white")
ax.axvline(growth_s.median(), color=BLACK, linestyle="--", linewidth=1.5,
           label=f"Median {growth_s.median():.1f}% / yr")
for state in SPOTLIGHT_STATES:
    if state in growth_s.index:
        val = growth_s[state]
        ax.barh(state.replace("_"," ").title(), val,
                color=YELLOW, edgecolor=BLACK, linewidth=1.5,
                label=f"{state.title()[:4]} {val:.1f}%")
ax.set_xlabel("Annualised Growth Rate (% / year)")
ax.set_title("Annualised Per-Pupil Expenditure Growth Rate by State\n"
             "Blue = above median  |  Red = below  |  Yellow = spotlight")
ax.legend(fontsize=9)
fig.tight_layout()
save(fig, "10a_annualised_growth_ranking.png")

# ── FIG 10-B  |  Slope divergence over rolling 10-yr windows ─────────────
print("── Fig 10-B  Rolling 10-yr trend slopes — divergence over time")

ROLL_W    = 10
year_list = sorted(df["YEAR"].unique())
window_starts = [y for y in year_list if y + ROLL_W <= year_list[-1]]

slope_matrix = {}
for state, grp in df.groupby("STATE"):
    grp  = grp.sort_values("YEAR").dropna(subset=[METRIC])
    rows = {}
    for ws in window_starts:
        sub = grp[(grp["YEAR"] >= ws) & (grp["YEAR"] < ws + ROLL_W)]
        if len(sub) < 5:
            continue
        sl, _, _, _, _ = _ols(sub["YEAR"].values.astype(float), sub[METRIC].values)
        rows[ws] = sl
    if rows:
        slope_matrix[state] = rows

slope_df = pd.DataFrame(slope_matrix).T  # states × window_starts

p10 = slope_df.quantile(0.10)
p25 = slope_df.quantile(0.25)
p75 = slope_df.quantile(0.75)
p90 = slope_df.quantile(0.90)
med = slope_df.median()

fig, ax = plt.subplots(figsize=(13, 7))
ax.fill_between(p10.index, p10, p90, alpha=0.15, color=BLUE, label="P10–P90")
ax.fill_between(p25.index, p25, p75, alpha=0.30, color=BLUE, label="P25–P75")
ax.plot(med.index, med.values, color=BLACK, linewidth=2.2, label="Median slope")

for state, label, color, marker in zip(SPOTLIGHT_STATES, SPOTLIGHT_LABELS,
                                        SPOTLIGHT_COLORS, SPOTLIGHT_MARKERS):
    if state in slope_df.index:
        row = slope_df.loc[state].dropna()
        ax.plot(row.index, row.values, linewidth=2, color=color,
                marker=marker, markersize=4, label=label)

ax.axhline(0, color=BLACK, linewidth=0.8, linestyle=":")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax.set_xlabel("Rolling Window Start Year")
ax.set_ylabel("OLS Slope ($ / year)")
ax.set_title("Rolling 10-Year OLS Trend Slopes — Per-Pupil Expenditure\n"
             "Fan band = cross-state dispersion; spotlight states shown individually")
ax.legend(ncol=2, fontsize=9)
fig.tight_layout()
save(fig, "10b_rolling_slope_divergence.png")

# ── FIG 10-C  |  Spotlight states — full decomposition side-by-side ───────
print("── Fig 10-C  Spotlight — HP trend + composite score dual-axis")

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for ax, state, label, color in zip(axes, SPOTLIGHT_STATES, SPOTLIGHT_LABELS, SPOTLIGHT_COLORS):
    sub    = spot[state].dropna(subset=[METRIC]).sort_values("YEAR")
    yrs    = sub["YEAR"].values
    vals   = sub[METRIC].values
    tr, cy = _hp_filter(vals, lam=6.25)

    ax2 = ax.twinx()

    ax.fill_between(yrs, vals, alpha=0.12, color=color)
    ax.plot(yrs, vals, color=color, linewidth=2.2, label="Expenditure")
    ax.plot(yrs, tr,   color=BLACK, linewidth=1.8, linestyle="--", label="HP Trend")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"${v/1000:.0f}k"))
    ax.set_ylabel("Exp / Pupil", color=color)

    score_sub = spot[state].dropna(subset=[SCORE]).sort_values("YEAR")
    if len(score_sub) >= 3:
        ax2.plot(score_sub["YEAR"], score_sub[SCORE], color=PURPLE,
                 linewidth=2, linestyle="-.", marker="s", markersize=5,
                 label="NAEP Score")
        ax2.set_ylabel("NAEP Composite", color=PURPLE)
        ax2.tick_params(axis="y", labelcolor=PURPLE)

    ax.set_title(label, color=color, fontweight="bold")
    ax.set_xlabel("Year")

    lines1, labs1 = ax.get_legend_handles_labels()
    lines2, labs2 = ax2.get_legend_handles_labels()
    ax.legend(lines1 + lines2, labs1 + labs2, fontsize=8, loc="upper left")

fig.suptitle("HP Trend + NAEP Score Overlay — Spotlight States\n"
             "Left axis: spending  |  Right axis (purple): test scores",
             fontsize=13, y=1.01)
fig.tight_layout()
save(fig, "10c_spotlight_trend_score_dual.png")

# ── FIG 10-D  |  Normalised index chart — everyone starts at 100 ──────────
print("── Fig 10-D  Indexed spending — all states (base=1993)")

BASE = 1993

fig, ax = plt.subplots(figsize=(14, 8))
for state, grp in df.groupby("STATE"):
    sub = grp.sort_values("YEAR").dropna(subset=[METRIC])
    base_val = sub[sub["YEAR"] == BASE][METRIC].values
    if len(base_val) == 0 or base_val[0] == 0:
        continue
    indexed = sub[sub["YEAR"] >= BASE][METRIC] / base_val[0] * 100
    idx_yrs = sub[sub["YEAR"] >= BASE]["YEAR"]
    sc = SPOTLIGHT_STATES
    if state in sc:
        i      = sc.index(state)
        ax.plot(idx_yrs, indexed, linewidth=2.8, color=SPOTLIGHT_COLORS[i],
                marker=SPOTLIGHT_MARKERS[i], markersize=4, zorder=5,
                label=SPOTLIGHT_LABELS[i])
    else:
        ax.plot(idx_yrs, indexed, linewidth=0.7, color="grey", alpha=0.3)

ax.axhline(100, color=BLACK, linewidth=1.2, linestyle="--", label=f"Base = {BASE}")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda v, _: f"{v:.0f}"))
ax.set_xlabel("Year")
ax.set_ylabel(f"Index  (100 = {BASE} value)")
ax.set_title(f"Per-Pupil Expenditure — Indexed to {BASE}=100\n"
             "Grey lines = all other states  |  Coloured = spotlight states")
ax.legend(fontsize=9)
fig.tight_layout()
save(fig, "10d_indexed_spending_all_states.png")


# =============================================================================
# DONE
# =============================================================================
print("\n" + "="*70)
print("TIME-SERIES NOTEBOOK COMPLETE")
print(f"All figures saved to:  outputs/figures/ts/")
print("="*70 + "\n")


SECTION 10 — HOW STATES TREND DIFFERENTLY

── Fig 10-A  Annualised growth rate distribution
  ✓  Saved  outputs/figures/ts/10a_annualised_growth_ranking.png
── Fig 10-B  Rolling 10-yr trend slopes — divergence over time
  ✓  Saved  outputs/figures/ts/10b_rolling_slope_divergence.png
── Fig 10-C  Spotlight — HP trend + composite score dual-axis
  ✓  Saved  outputs/figures/ts/10c_spotlight_trend_score_dual.png
── Fig 10-D  Indexed spending — all states (base=1993)
  ✓  Saved  outputs/figures/ts/10d_indexed_spending_all_states.png

TIME-SERIES NOTEBOOK COMPLETE
All figures saved to:  outputs/figures/ts/

